In [ ]:
from google import genai
import pandas as pd
import json

# Configure the API key
client = genai.Client(api_key="")

# Load your datasets into strings
try:
    broker_a_data = pd.read_csv("BrokerA_dataset.csv").to_csv(index=False)
    broker_b_data = pd.read_csv("BrokerB_dataset.csv").to_csv(index=False)
    social_data = pd.read_csv("SocialPlatformX_dataset.csv").to_csv(index=False)
except FileNotFoundError:
    print("Error: Make sure all three CSV files are in the same directory.")
    exit()

# The prompt is the key – see the next section for the full prompt text
prompt_template = """
You are an expert financial fraud analyst. Your task is to detect pump-and-dump schemes by analyzing trading and social media data.

You will be given three datasets in CSV format:
1.  **BrokerA_dataset.csv**: Trading data from Broker A, including an anonymized_trader_id.
2.  **BrokerB_dataset.csv**: Trading data from Broker B, including an anonymized_trader_id.
3.  **SocialPlatformX_dataset.csv**: Social media activity data.

**Follow these steps precisely:**

1.  **Combine Data**: Mentally combine the trading data from Broker A and Broker B.
2.  **Identify Social Hype**: Scan the social media data for timestamps where the 'activity_level' is greater than 30000 for a stock. Note these as "hype events."
3.  **Identify Trading Anomalies**: Scan the combined trading data for timestamps where the 'trade_volume' is greater than 500000. Note these as "trading anomalies."
4.  **Correlate Events**: Find instances where a "trading anomaly" for a specific stock occurs within 15 minutes *after* a "hype event" for the same stock.
5.  **Generate Alerts**: For each correlated event you find, generate a detailed alert. Identify the `anonymized_trader_id`s involved in the anomalous trades.

**Output Format:**
List each trading anomalies
List each hype event
List each correlation event if exist.
Do not include introductory remarks or topics, just give the findings in continuous text.

---
**DATASETS:**

**BrokerA_dataset.csv:**
```csv
{broker_a_csv}

**BrokerB_dataset.csv:**
```csv
{broker_b_csv}

**SocialPlatformX_dataset.csv:**
```csv
{social_csv}
"""

# Insert the data into the prompt
full_prompt = prompt_template.format(
    broker_a_csv=broker_a_data[:2500],
    broker_b_csv=broker_b_data[:2500],
    social_csv=social_data[:2500]
)

# Call the Gemini API
# model = genai.GenerativeModel('gemini-2.5-flash')
# response = model.generate_content(full_prompt)
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=full_prompt
)
# stream = client.models.generate_content_stream(
    # model="gemini-2.5-flash",
    # contents=full_prompt,
# )

# Print the model's analysis
print(response)
# alerts_json = response.to_dict()
with open("result.json", "w", encoding="utf-8") as f:
    json.dump(response, f, indent=4)
# for event in stream:
#     # Every event has candidates (model responses)
#     for cand in event.candidates or []:
#         if cand.content and cand.content.parts:
#             for part in cand.content.parts:
#                 if hasattr(part, "text") and part.text:
#                     print(part.text, end="", flush=True)

sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""**Trading Anomalies:**
*   Stock: GME, Timestamp: 2021-01-20 09:33:15.790837248-05:00, Trade Volume: 2607556, Anonymized Trader ID: user-h3i8
*   Stock: GME, Timestamp: 2021-01-20 16:14:20.841945856-05:00, Trade Volume: 3171136, Anonymized Trader ID: user-d9f2
*   Stock: GME, Timestamp: 2021-01-20 17:34:52.934853120-05:00, Trade Volume: 1702850, Anonymized Trader ID: user-d9f2

**Hype Events:**
*   Stock: GME, Timestamp: 2021-01-20 16:39:31.450308864-05:00, Activity Level: 144016

**Correlation Events:**
No correlated events found where a trading anomaly for a specific stock occurs within 15 minutes *after* a hype event for the same stock."""
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='D2C-aLSmAcCmmtkPvdKX-QI' usag

TypeError: Object of type GenerateContentResponse is not JSON serializable